In [1]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from math import sin, pi
import ipdb
import re
import csv
import datetime as dt
from copy import deepcopy
from glob import glob
import ast
from collections import Counter
from IPython.core.debugger import set_trace
import datasets

tqdm.pandas()

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('chained_assignment',None)

# Set random seeds for reproducibility on a specific machine
random.seed(1)
np.random.seed(1)
np.random.RandomState(1)

RandomState(MT19937) at 0x1285FB040

In [16]:
data = datasets.load_dataset('csv', data_files='../data/test_data.tsv', sep='\t', index_col=None, quoting=csv.QUOTE_NONE, escapechar='\\')['train']

def wp_to_description(row):
    if ((row['win_prob']>=0) and (row['win_prob']<0.1)):
        row['game_state'] = row['opp'].title() + " are extremely likely to win."
    elif ((row['win_prob']>=0.1) and (row['win_prob']<0.25)):
        row['game_state'] = row['opp'].title() + " are likely to win."
    elif ((row['win_prob']>=0.25) and (row['win_prob']<0.48)):
        row['game_state'] = row['opp'].title() + " are slightly likely to win."
    elif ((row['win_prob']>=0.52) and (row['win_prob']<0.75)):
        row['game_state'] = row['team'].title() + " are slightly likely to win." 
    elif ((row['win_prob']>=0.75) and (row['win_prob']<0.9)):
        row['game_state'] = row['team'].title() + " are likely to win."
    elif ((row['win_prob']>=0.9) and (row['win_prob']<=1)):
        row['game_state'] = row['team'].title() + " are extremely likely to win."   
    else:
        row['game_state'] = "Both teams are equally likely to win." 
    return row

data = data.map(wp_to_description)

with open('prompts/llama-wp-ling.txt') as f:
    instructions = f.read()
        
def create_input_context(row):
    '''
    Takes an input row, and makes a long string with instruction, comment, in-group team, out-group team, as well as win probability.
    '''  
    
    row['input'] = (
        "COMMENT: " + row["tokenized_comment"] + "\n" +
        "IN-GROUP: " + row['team'].title() + "\n" +
        "OUT-GROUP: " + row['opp'].title() + "\n" +
        "GAME STATE: " + row['game_state'] + "\n"
        "REF_EXPRESSIONS: "
    )
    
    row['output'] = (
         str(row['ref_expressions']) + "\n"
        "EXPLANATION: " + str(row['explanation']) + "\n"
        "TARGET: " + str(row['tagged_comment'])
    )
    
    row['instruction'] = instructions  
    row['weight'] = [np.round(sin(row['win_prob']*pi), 3)+0.5]
    return row

data = data.map(create_input_context)
train_data = data.filter(lambda row: row['split']=='train').select_columns(['instruction', 'input', 'output'])
eval_data = data.filter(lambda row: row['split']=='test').select_columns(['instruction', 'input', 'output'])

Map:   0%|          | 0/1499 [00:00<?, ? examples/s]

Map:   0%|          | 0/1499 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1499 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1499 [00:00<?, ? examples/s]

In [17]:
train_json = train_data.to_pandas().to_json(orient='records', indent=4, force_ascii=False)
with open('data/all_train_ling_wp.json', 'w') as f:
    f.write(train_json)
    
eval_json = eval_data.to_pandas().to_json(orient='records', indent=4, force_ascii=False)
with open('data/all_eval_ling_wp.json', 'w') as f:
    f.write(eval_json)

In [26]:
with open('model-outputs/ref-inputs.txt', 'w') as f:
    for line in eval_data['input']:
        f.write(line[9:-18].replace('\n', ' ') + "\n")